In [166]:
#ライブラリをインポート
import os #OSに依存する様々な機能を利用するためのモジュール(ファイルやディレクトリ操作など)
import re #正規表現を利用するためのモジュール
import csv  #csvファイルを扱うためのモジュール
import math #数学的計算のためのモジュール
import matplotlib.pyplot as plt #グラフ描画のためのモジュール
import numpy as np  #多次元配列計算のためのモジュール
import pandas as pd #データフレームを扱うためのモジュール
from sklearn.model_selection import train_test_split  #データをトレーニング用とテスト用に分けるためのモジュール
from sklearn.linear_model import LinearRegression #線型回帰
from sklearn.svm import SVC #サポートベクターマシン
from sklearn.ensemble import RandomForestClassifier #ランダムフォレスト
from sklearn.metrics import accuracy_score  #機械学習モデルの性能評価のためのモジュール

In [188]:
#定数を定義
BINS = 4000  #ヒストグラムのビンの数
EPSILON = .00001  #スムージングパラメータ
UPPER_LIMIT = 1.1 #静止区間の上限
LOWER_LIMIT = 0.9 #静止区間の加減
STATIONARY_INTERVALS = 5  #静止区間除去のサンプルの間隔(静止区間が何サンプル連続したら除去するか)
TRAIN_SIZE = 0.8  #ランダムフォレストのトレーニングデータの割合

In [168]:
#ディレクトリ内のデータセットのファイル名と周波数を取得する関数
def get_Hz_and_filename(path: str) -> list[int, str]:
    filename = os.listdir(path) #引数のパスのディレクトリの中のファイル名一覧を取得
    Hz_and_filename=[]  #ファイル名と周波数を格納するリストを宣言

    for file in filename:
        Hz = re.search(r'\d+', file)    #正規表現を用いてファイル名の中で一番最初に出てくる数字(周波数)を取得
        if Hz:  #数字の入っていないファイル名があるとエラーを吐くので、このif文でチェックする
            Hz_and_filename.append([int(Hz.group(0)), file])    #ファイル名と周波数を格納

    return Hz_and_filename

In [169]:
#ファイル名と周波数を分けて出力する関数
def divide_Hz_and_filename(Hz_and_filename: list[int, str]) -> tuple[list[int], list[str]]:
    Hz = []
    filename = []
    for row in Hz_and_filename:
      Hz.append(row[0])
      filename.append(row[1])

    return Hz, filename

In [170]:
#加速度データのCSVファイルから3軸加速度を取得する関数
def get_acceleration(filename: str) -> tuple[list[float], list[float], list[float]]:
    AccX, AccY, AccZ = [], [], []
    with open(filename) as f:
        reader = csv.reader(f)
        for row in reader:
            AccX.append(float(row[2]))
            AccY.append(float(row[3]))
            AccZ.append(float(row[4]))

    return AccX, AccY, AccZ

In [171]:
#静止区間を除去する関数
def remove_stationary_intervals(AccX: list[float], AccY: list[float], AccZ: list[float]) -> list[float]:
    #各軸の加速度の平均を求める
    AvgAccX = sum(AccX) / len(AccX)
    AvgAccY = sum(AccY) / len(AccY)
    AvgAccZ = sum(AccZ) / len(AccZ)

    AvgResultantAcc = math.sqrt(AvgAccX ** 2 + AvgAccY ** 2 + AvgAccZ ** 2) #重力加速度の推定値=合成加速度の平均を求める

    ResultantAcc = [math.sqrt(x ** 2 + y ** 2 + z ** 2) for x, y, z in zip(AccX, AccY, AccZ)]   #各時刻の合成加速度を求める

    #各時刻の合成加速度から静止区間(重力加速度の推定値に近い値が一定以上以上連続している区間)を除去する
    i=0 #ループ変数
    counter = 0 #静止区間がSTATIONARY_INTERVALS分続いているかをカウントする変数
    while i < len(ResultantAcc):
        if AvgResultantAcc * LOWER_LIMIT < ResultantAcc[i] < AvgResultantAcc * UPPER_LIMIT:   #平均のLOWER_LIMIT倍~UPPER_LIMIT倍の範囲を調べる
            counter += 1    #範囲内ならカウントを増やす
            if counter == STATIONARY_INTERVALS: #カウントがSTATIONARY_INTERVALSに達したらその区間を削除
                del ResultantAcc[i+1-STATIONARY_INTERVALS:i+1]    #スライスでは選択範囲の開始位置startと終了位置stopを[start:stop]のように書くとstart <= x < stopの範囲が選択される #start番目の値は含まれるがstop番目の値は含まれない
                counter = 0 #カウンターをリセット
                i -= STATIONARY_INTERVALS   #削除した分インデックスがズレるので補正する
        else:
            counter = 0 #カウンターをリセット
        i += 1

    return ResultantAcc  #静止区間を除去した後のリストを返す

In [172]:
#連続する2サンプルの差分を取る関数
def differences_of_acceleration(ResultantAcc: list[float]) -> list[float]:
    index = 0
    DifferenceAcc = []
    for dif in ResultantAcc[:-1]:
        DifferenceAcc.append(math.fabs(ResultantAcc[index + 1]*100000 - ResultantAcc[index]*100000))    #100000倍して誤差を取る
        index += 1

    return DifferenceAcc

In [173]:
#KLダイバージェンス関数 #引数として与える2つの分布は非負の値の集合でなければならないことに注意
def KL_divergence(a: list[float], b: list[float]) -> float:
    min_value = min(min(a), min(b)) #a,bの最小値の小さい方
    max_value = max(max(a), max(b)) #a,bの最大値の大きい方

    #a,bのヒストグラムを作成し、同じ数のビンで区切る
    a_hist, _ = np.histogram(a, bins=BINS, range=(min_value, max_value))
    b_hist, _ = np.histogram(b, bins=BINS, range=(min_value, max_value))

    #正規化する(確率分布に変換する、合計を1にする)ために全合計で割る
    a_hist = (a_hist + EPSILON) / a_hist.sum()
    b_hist = (b_hist + EPSILON) / b_hist.sum()

    #KLダイバージェンスの値を返す
    return np.sum([ai * np.log(ai / bi) for ai, bi in zip(a_hist, b_hist)])

In [174]:
#JSダイバージェンス関数 #引数として与える2つの分布は非負の値の集合でなければならないことに注意
def JS_divergence(a: list[float], b: list[float]) -> float:
    min_value = min(min(a), min(b)) #a,bの最小値の小さい方
    max_value = max(max(a), max(b)) #a,bの最大値の大きい方

    #a,bのヒストグラムを作成し、同じ数のビンで区切る
    a_hist, _ = np.histogram(a, bins=BINS, range=(min_value, max_value))
    b_hist, _ = np.histogram(b, bins=BINS, range=(min_value, max_value))

    #正規化する(確率分布に変換する、合計を1にする)ために全合計で割る
    a_hist = (a_hist + EPSILON) / a_hist.sum()
    b_hist = (b_hist + EPSILON) / b_hist.sum()

    #2つの分布の平均値を求める
    mean_hist = (a_hist + b_hist) / 2.0

    #平均とそれぞれの分布のKLダイバージェンスを算出
    kl_a = np.sum([ai * np.log(ai / bi) for ai, bi in zip(a_hist, mean_hist)])
    kl_b = np.sum([ai * np.log(ai / bi) for ai, bi in zip(b_hist, mean_hist)])

    #JSダイバージェンスの値を返す
    return (kl_a + kl_b) / 2.0

In [175]:
#データフレームの各行の中で2番目に小さい値が格納されている場所を調べる関数(最小値は同じ確率分布同士の0.0)
def get_index_and_columns_of_second_smallest(df: pd.DataFrame) -> list[str, str]:
    index_and_columns_of_second_smallest = []  #データフレームの中で2番目に小さい値が格納されている場所のインデックス名とカラム名を格納する変数
    for i in range(len(df)):
        sorted_row = df.iloc[i].sort_values()   #.ilocでデータフレームの要素を行、列の番号の添字で指定する    #各行の要素を昇順に並び替える
        second_smallest_columns = sorted_row.index[1] #各行の2番目に小さい値が格納されているカラム[1]の名前を取得
        #second_smallest_label = df.columns.get_loc(second_smallest_index)
        index_and_columns_of_second_smallest.append((df.index[i], second_smallest_columns))    #インデックスとカラムのラベル名の組を二次元配列に追加
    return index_and_columns_of_second_smallest

In [176]:
#推定精度を算出する巻数
def calculate_accuracy(index_and_columns_of_second_smallest: list[str, str]) -> tuple[float, list[int]]:
    counter = 0
    error_index_list = []
    for i in range(len(index_and_columns_of_second_smallest)):
        #インデックスとカラムのラベル名が同じならばカウンターを1増やす
        if index_and_columns_of_second_smallest[i][0] == index_and_columns_of_second_smallest[i][1]:
            counter += 1
        else:
            error_index_list.append(i)
            print(f"間違ってるやつは{i}番目の{index_and_columns_of_second_smallest[i][0]}と{index_and_columns_of_second_smallest[i][1]}です")

    return (counter / len(index_and_columns_of_second_smallest)) * 100, error_index_list  #精度を100分率で返す

In [177]:
#入力された加速度の差分のリストからヒストグラムを作る関数
def create_histogram(DifferenceAcc_list: list[float]) -> np.histogram:
    min_value = min(map(lambda x:max(x), DifferenceAcc_list))   #入力されたリストの中で最も小さい数
    max_value = max(map(lambda x:max(x), DifferenceAcc_list))   #入力されたリストの中で最も大きい数

    DifferenceAcc_hist = np.zeros((len(DifferenceAcc_list), BINS), dtype=float)
    print(DifferenceAcc_hist[0])

    for i in range(len(DifferenceAcc_list)):
        DifferenceAcc_hist[i], _ = np.histogram(DifferenceAcc_list[i], bins=BINS, range=(min_value, max_value)) #ヒストグラムを作成し、同じ数のビンで区切る
        DifferenceAcc_hist[i] = (DifferenceAcc_hist[i] + EPSILON) / DifferenceAcc_hist[i].sum()     #正規化する(確率分布に変換する、合計を1にする)ために全合計で割る
    return DifferenceAcc_hist

In [178]:
#KLダイバージェンスとJSダイバージェンス算出の一連の流れを自動化した関数
def KL_and_JS(path: str):
    Hz_and_filename = get_Hz_and_filename(path)
    Hz_and_filename.sort(reverse=True)  #周波数の大きい順にソート
    Hz, filename = divide_Hz_and_filename(Hz_and_filename)
    Hz = [str(hz) + "Hz" for hz in Hz]  #周波数の値+"Hz"のリストを作りデータフレームのラベルに用いる

    #使う変数を宣言
    AccX, AccY, AccZ = [], [], []
    ResultantAcc = []
    DifferenceAcc_list = []
    resultKLD = [[0.0 for j in range(len(filename))] for i in range(len(filename))]  # resultKLDの要素を0.0で初期化
    resultJSD = [[0.0 for j in range(len(filename))] for i in range(len(filename))]  # resultKLDの要素を0.0で初期化
    error_index_list = []

    #各データセットからデータを読み込み静止区間を除去したものを二次元配列に格納
    for i in filename:
        AccX, AccY, AccZ = get_acceleration(path+i)
        ResultantAcc = remove_stationary_intervals(AccX, AccY, AccZ)
        DifferenceAcc_list.append(differences_of_acceleration(ResultantAcc))

    #KLダイバージェンスの値を格納
    for i in range(len(filename)):
        for j in range(len(filename)):
            resultKLD[i][j] = KL_divergence(DifferenceAcc_list[i], DifferenceAcc_list[j])

    #JSダイバージェンスの値を格納
    for i in range(len(filename)):
        for j in range(len(filename)):
            resultJSD[i][j] = JS_divergence(DifferenceAcc_list[i], DifferenceAcc_list[j])

    #結果を出力
    df_KLD = pd.DataFrame(resultKLD, index=Hz, columns=Hz)
    display(df_KLD)
    accuracyKLD, error_index_list = calculate_accuracy(get_index_and_columns_of_second_smallest(df_KLD))
    for i in range(len(error_index_list)):
        print(filename[error_index_list[i]])
    print(f"KLダイバージェンスによる推定精度は{accuracyKLD}%です")

    df_JSD = pd.DataFrame(resultJSD, index=Hz, columns=Hz)
    display(df_JSD)
    accuracyJSD, error_index_list = calculate_accuracy(get_index_and_columns_of_second_smallest(df_JSD))
    for i in range(len(error_index_list)):
        print(filename[error_index_list[i]])
    print(f"JSダイバージェンスによる推定精度は{accuracyJSD}%です")

In [179]:
#ランダムフォレストによる機械学習モデル構築と性能評価までを自動化した関数
def random_forest(path: str):
    Hz_and_filename = get_Hz_and_filename(path)
    Hz_and_filename.sort(reverse=True)  #周波数の大きい順にソート
    Hz, filename = divide_Hz_and_filename(Hz_and_filename)

    #使う変数を宣言
    AccX, AccY, AccZ = [], [], []
    ResultantAcc = []
    DifferenceAcc_list = []

    #各データセットからデータを読み込み静止区間を除去したものを二次元配列に格納
    for i in filename:
        AccX, AccY, AccZ = get_acceleration(path+i)
        ResultantAcc = remove_stationary_intervals(AccX, AccY, AccZ)
        DifferenceAcc_list.append(differences_of_acceleration(ResultantAcc))

    DifferenceAcc_hist = create_histogram(DifferenceAcc_list)
    x_train, x_test, y_train, y_test = train_test_split(DifferenceAcc_hist, Hz, train_size = TRAIN_SIZE, shuffle = True)

    # 学習する
    clf = RandomForestClassifier(random_state=1234)
    clf.fit(x_train, y_train)
    y_pred = clf.predict(x_test)
    print("正解率 = ", accuracy_score(y_test, y_pred))

In [180]:
path = "all_walk_data/"

In [187]:
KL_and_JS(path)

,100Hz,100Hz,100Hz,100Hz,100Hz,100Hz,100Hz,100Hz,100Hz,100Hz,...,10Hz,10Hz,10Hz,10Hz,10Hz,10Hz,10Hz,10Hz,10Hz,10Hz
100Hz,0.000000,0.102401,0.450648,0.223768,0.074642,0.135246,0.396460,0.385656,0.442086,0.387380,...,0.164917,1.120338,1.000631,1.060946,1.365631,0.417551,0.243506,1.185094,1.108083,0.989978
100Hz,0.130670,0.000000,0.200497,0.252220,0.165030,0.054336,0.183811,0.333799,0.383754,0.326759,...,0.142594,1.385051,1.227512,1.146138,1.649955,0.472268,0.205295,1.492908,1.450191,1.238161
100Hz,0.479147,0.192045,0.000000,0.379365,0.529372,0.121658,0.024878,0.395920,0.404630,0.360854,...,0.295444,1.959388,1.759419,1.397186,2.263692,0.756648,0.306043,2.159676,2.043234,1.808182
100Hz,0.102380,0.101411,0.334356,0.000000,0.231317,0.107083,0.281438,0.086128,0.084489,0.074077,...,0.283858,1.531582,1.377153,1.389383,1.786793,0.648565,0.385082,1.646026,1.575018,1.390696
100Hz,0.275475,0.330681,0.650540,0.916467,0.000000,0.316286,0.676862,1.394577,1.340471,1.138547,...,0.123858,0.788175,0.695600,0.718892,0.998415,0.230819,0.158564,0.856932,0.819199,0.670605
100Hz,0.188975,0.072935,0.153951,0.229948,0.222637,0.000000,0.135448,0.328874,0.337625,0.283453,...,0.151934,1.463348,1.300122,1.150335,1.729080,0.507242,0.201280,1.612256,1.503066,1.330052
100Hz,0.424505,0.159089,0.014772,0.317105,0.485841,0.092379,0.000000,0.325106,0.338960,0.306727,...,0.273243,1.899180,1.713341,1.368683,2.208736,0.726334,0.283623,2.102321,1.981283,1.750118
100Hz,0.175560,0.105492,0.294619,0.053925,0.325113,0.129299,0.251342,0.000000,0.054105,0.020069,...,0.328126,1.902438,1.548256,1.543267,1.976150,0.746061,0.460379,1.829607,3.351959,1.578744
100Hz,0.185927,0.127237,0.318183,0.053952,0.334062,0.143100,0.273952,0.037866,0.000000,0.020653,...,0.347960,1.912353,1.563573,1.560660,1.982400,0.766316,0.476001,1.848111,3.103393,1.593824
100Hz,0.195558,0.128774,0.310500,0.051817,0.343652,0.141461,0.266192,0.026875,0.024205,0.000000,...,0.352185,1.763856,1.576915,1.566277,2.006868,0.774809,0.477900,1.868882,1.854915,1.597159


間違ってるやつは0番目の100Hzと50Hzです
間違ってるやつは4番目の100Hzと50Hzです
間違ってるやつは5番目の100Hzと50Hzです
間違ってるやつは13番目の50Hzと100Hzです
間違ってるやつは17番目の50Hzと100Hzです
間違ってるやつは19番目の50Hzと100Hzです
間違ってるやつは22番目の50Hzと100Hzです
間違ってるやつは23番目の50Hzと100Hzです
間違ってるやつは25番目の10Hzと50Hzです
間違ってるやつは26番目の10Hzと100Hzです
walk100Hz-20230303-111623870.csv
walk100Hz-20230228-195310844.csv
walk100Hz-20230227-202328538.csv
walk50Hz-20230302-165451196.csv
walk50Hz-20230227-202335920.csv
walk50Hz-0930-0756.csv
walk50Hz-0803-1229.csv
walk50Hz-0803-1126.csv
walk10Hz-20230302-165445244.csv
walk10Hz-20230302-115939846.csv
KLダイバージェンスによる推定精度は72.22222222222221%です


,100Hz,100Hz,100Hz,100Hz,100Hz,100Hz,100Hz,100Hz,100Hz,100Hz,...,10Hz,10Hz,10Hz,10Hz,10Hz,10Hz,10Hz,10Hz,10Hz,10Hz
100Hz,0.000000,0.022277,0.106893,0.028227,0.021662,0.032893,0.094409,0.049234,0.053589,0.056872,...,0.044245,0.281974,0.261806,0.272629,0.328775,0.119210,0.065664,0.284488,0.269566,0.253252
100Hz,0.022277,0.000000,0.043511,0.025937,0.040852,0.007593,0.036753,0.029127,0.035001,0.035881,...,0.034753,0.314709,0.292609,0.283996,0.358147,0.129181,0.051905,0.322620,0.311246,0.287408
100Hz,0.106893,0.043511,0.000000,0.080013,0.126845,0.028377,0.002243,0.073235,0.077776,0.075287,...,0.078542,0.396491,0.372999,0.339274,0.433919,0.197902,0.084122,0.413592,0.398133,0.375887
100Hz,0.028227,0.025937,0.080013,0.000000,0.069611,0.027673,0.068249,0.013567,0.013006,0.012406,...,0.081514,0.375242,0.350341,0.349404,0.417284,0.185998,0.107617,0.390652,0.372854,0.349911
100Hz,0.021662,0.040852,0.126845,0.069611,0.000000,0.051939,0.117768,0.093811,0.097944,0.101817,...,0.020398,0.185913,0.174272,0.180052,0.232939,0.057978,0.031936,0.186742,0.177501,0.158159
100Hz,0.032893,0.007593,0.028377,0.027673,0.051939,0.000000,0.022182,0.034338,0.038149,0.038079,...,0.037300,0.326843,0.303513,0.288647,0.368260,0.137921,0.050596,0.340320,0.323728,0.302457
100Hz,0.094409,0.036753,0.002243,0.068249,0.117768,0.022182,0.000000,0.063472,0.068091,0.065811,...,0.074564,0.393619,0.370210,0.339198,0.432449,0.193724,0.080430,0.410922,0.395335,0.372702
100Hz,0.049234,0.029127,0.073235,0.013567,0.093811,0.034338,0.063472,0.000000,0.005906,0.004203,...,0.093079,0.405562,0.375347,0.370704,0.439349,0.207370,0.124903,0.415734,0.419152,0.377905
100Hz,0.053589,0.035001,0.077776,0.013006,0.097944,0.038149,0.068091,0.005906,0.000000,0.002791,...,0.099154,0.412632,0.381713,0.376534,0.444452,0.213661,0.129830,0.423255,0.423669,0.385481
100Hz,0.056872,0.035881,0.075287,0.012406,0.101817,0.038079,0.065811,0.004203,0.002791,0.000000,...,0.101352,0.412528,0.386242,0.380312,0.450171,0.217969,0.131891,0.429462,0.414479,0.388545


間違ってるやつは0番目の100Hzと50Hzです
間違ってるやつは4番目の100Hzと50Hzです
間違ってるやつは13番目の50Hzと100Hzです
間違ってるやつは22番目の50Hzと100Hzです
間違ってるやつは25番目の10Hzと50Hzです
間違ってるやつは31番目の10Hzと50Hzです
walk100Hz-20230303-111623870.csv
walk100Hz-20230228-195310844.csv
walk50Hz-20230302-165451196.csv
walk50Hz-0803-1229.csv
walk10Hz-20230302-165445244.csv
walk10Hz-20230227-202332703.csv
JSダイバージェンスによる推定精度は83.33333333333334%です


In [182]:
random_forest(path)

[0. 0. 0. ... 0. 0. 0.]
正解率 =  0.25


In [189]:
Hz_and_filename = get_Hz_and_filename(path)
Hz_and_filename.sort(reverse=True)  #周波数の大きい順にソート

In [192]:
Hz_and_filename

[[100, 'walk100Hz-20230303-111623870.csv'],
 [100, 'walk100Hz-20230302-165446613.csv'],
 [100, 'walk100Hz-20230302-115943109.csv'],
 [100, 'walk100Hz-20230301-203316681.csv'],
 [100, 'walk100Hz-20230228-195310844.csv'],
 [100, 'walk100Hz-20230227-202328538.csv'],
 [100, 'walk100Hz-20230227-194442583.csv'],
 [100, 'walk100Hz-0930-0756.csv'],
 [100, 'walk100Hz-0809-1000.csv'],
 [100, 'walk100Hz-0809-0900.csv'],
 [100, 'walk100Hz-0803-1229.csv'],
 [100, 'walk100Hz-0803-1126.csv'],
 [50, 'walk50Hz-20230303-111631944.csv'],
 [50, 'walk50Hz-20230302-165451196.csv'],
 [50, 'walk50Hz-20230302-115947707.csv'],
 [50, 'walk50Hz-20230301-203318690.csv'],
 [50, 'walk50Hz-20230228-195313062.csv'],
 [50, 'walk50Hz-20230227-202335920.csv'],
 [50, 'walk50Hz-20230227-194450631.csv'],
 [50, 'walk50Hz-0930-0756.csv'],
 [50, 'walk50Hz-0809-1000.csv'],
 [50, 'walk50Hz-0809-0900.csv'],
 [50, 'walk50Hz-0803-1229.csv'],
 [50, 'walk50Hz-0803-1126.csv'],
 [10, 'walk10Hz-20230303-111627793.csv'],
 [10, 'walk10Hz-

In [ ]:
get_acceleration()